In [222]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

#create the scroll_down function to scroll down the page
def scroll_down():
    btn_show = driver.find_element_by_css_selector('.button.span-100')
    btn_show.click()
    last =0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        html = BeautifulSoup(driver.page_source)
        curr = len(html.find_all('groupCard--photo nametag-photo'))
        if curr > last:
            last = curr
        else:
            break


#create driver
driver = webdriver.Chrome(u'C:\\Users\\Lydia Kan\\Desktop\\chromed\\chromedriver')
driver.implicitly_wait(3)

#get to the website main page
driver.get('https://www.meetup.com/')
#get the page sour as html
html= driver.page_source
#getting the soup from the source code
temp_soup = BeautifulSoup(html)

#collacting urls for each categor
list_category = temp_soup.find_all('li', {'class': 'gridList-item'})
result = []
for tag in list_category:
    url = tag.a.get('href')
    name = tag.h4.text
    result.append({'url': url, 'name': name})

#collacting urls for each event
result_event = []
for link in result:
    driver.get(link['url'])
    try: 
        scroll_down()
    except NoSuchElementException as e:
        print e.message
    html_sub = driver.page_source
    sub_soup = BeautifulSoup(html_sub)
    list_event = sub_soup.find_all('a', {'class': 'display-none'})
    
    for tag in list_event:
        url = tag.get('href')
        name = tag.text
        result_event.append({'url':url, 'name':name, 'category':link['name']})

#iterate each event pages

C:\Users\Lydia Kan\Anaconda2\lib\site-packages\ipykernel\__main__.py:51: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [271]:
#collacting data for each event page
final_list=[]

for link in result_event:
    
    driver.get(link['url'])
   
    driver.implicitly_wait(2)
   
    html_main = driver.page_source
    main_soup =BeautifulSoup(html_main)
    
    event_data={}
    #panal data
    group_panal = main_soup.find_all('ul', {'class':'paddedList small margin-bottom'})
    #scrapping each variable in the panal 
    lis = group_panal[0].find_all('li')
    try:
        #assign member's names and numbers to dictionary due to each event's member's names are different
        group, member = map(lambda x: x.string.strip(), lis[0].find_all('span'))
        event_data['group'] = group
        event_data['member'] = member
    except AttributeError as e:
        print e.message
    
    except IndexError as e:
        print e.message
    
    for li in lis[1:]:
        spans = li.findAll('span')
        try:
            key, value = map(lambda x: x.string.strip(), spans)
            event_data[key] = value
        except AttributeError as e:
            print e.message

    
    location = main_soup.find_all('span', {'class':'locality'})[0].get_text()
    event_data['Location'] = location
    
    region = main_soup.find_all('span', {'class':'region'})[0].get_text()
    event_data['Region'] = region
    
    date_panal = main_soup.find_all('div', {'class': 'small margin-bottom'})[0].get_text()
    founded_date = date_panal.split('\n')[2].replace(',','')
    event_data['FoundedDate'] = founded_date
    
    
    try:
        up_e_name = main_soup.find_all('a',{'class':'hoverLink omnCamp omnrv_rv13'})[0].get_text()
        event_data['UpcommingEventName'] = up_e_name
    except IndexError as e:
        print e.message
        
    #if not ('UpcommingEventName' in event_data):
        #try:
            #up_e_name = main_soup.find_all('h3',{'class':'big'})[0].get_text()
            #event_data['UpcommingEventName'] = up_e_name
        #except:
            #print 'Still not'
    
    try:
        up_e_timep= main_soup.find_all('li',{'class':'dateTime'})[0].get_text()
        up_e_date = up_e_timep.split('\n')[2]
        up_e_time = up_e_timep.split('\n')[3]
        event_data['UpcommingEventTime'] = up_e_time
        event_data['UpcommingEventDate'] = up_e_date
    except IndexError as e:
        print e.message
        
#    if main_soup.find_all('li',{'class':'dateTime'})==[]:
#        try:
#            up_e_timep= main_soup.find_all('p', {'class':'margin-none'})[0].get_text()
#            event_data['UpcommingEventTime'] = up_e_timep
#            tmp = re.split('[, ]', up_e_timep)
#            print tmp
#            up_e_date = tmp[0][:3]+' '+tmp[2][:3]+' '+tmp[3]
#            event_data['UpcommingEventDate']=up_e_date
#            print up_e_date
#            up_e_time= tmp[6]+' '+tmp[7]
#            event_data['UpcommingEventTime']= up_e_time
#            
#        except IndexError as e:
#            pass
    
        
    try:
        up_e_num = main_soup.find_all('li',{'class':'rsvpCount event-count rsvp-meta-subtle-link small'})[0].get_text()
        up_e_num = up_e_num.split('\n')[2]
        event_data['UpcommingEventGoing'] = up_e_num
    except IndexError as e:
        print e.message 
        
#    if not ('UpcommingEventGoing' in event_data):
#        try:
#            up_e_num = main_soup.find_all('div',{'class':'gutter-left lastUnit'})[0].get_text()
#            tmp= re.split('[, ]', up_e_num)
#            print tmp
#            up_e_num = tmp[0][1:3]+' going'
#            event_data['UpcommingEventGoing'] = up_e_num
#            
#        except:
#            print 'Still not'
        
    #add data from link into dictionary
    event_data['name'] = link['name']
    event_data['Category']= link['category']
    final_list.append(event_data)

C:\Users\Lydia Kan\Anaconda2\lib\site-packages\ipykernel\__main__.py:35: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'


C:\Users\Lydia Kan\Anaconda2\lib\site-packages\ipykernel\__main__.py:53: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
C:\Users\Lydia Kan\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
C:\Users\Lydia Kan\Anaconda2\lib\site-packages\ipykernel\__main__.py:92: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of 

C:\Users\Lydia Kan\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has n

C:\Users\Lydia Kan\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


list index out of range
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
list index out of range
list index out of range
list index out of range
'NoneType' object 

In [272]:
#convert data to dataframe
df = pd.DataFrame(final_list)

In [270]:
#save data as csv
df.to_csv('meetupfv.csv',encoding='utf-8')

df1 = pd.DataFrame(result_event)
df1.to_csv('meetupurl.csv', encoding='utf-8')

In [154]:
len(result_event)

#     group_mem = group_panal[0].find_all('li')[0].get_text()
#     group_mem= int(group_mem.split('\n')[2].replace(',',''))
#     group_reviews = group_panal[0].find_all('li')[1].get_text()
#     group_reviews = int(group_reviews.split('\n')[3])
#     upcoming_number = group_panal[0].find_all('li')[2].get_text()
#     upcoming_number = int(upcoming_number.split('\n')[3])
#     past_number = group_panal[0].find_all('li')[3].get_text()
#     past_number = int(past_number.split('\n')[3].replace(',',''))

#str(re.sub(u'\u2019', u'', df['UpcommingEventName'].iloc[8]))
#map(lambda x: str(re.sub(r'\u2019', '',x)), df['UpcommingEventName'].iloc[9]).to_csv('text.txt',  encoding = 'utf-8')

2435